# Tracking Methods: CPOL

This tutorial/demo illustrates how THUNER can be applied to [CPOL](https://www.openradar.io/research-radars/cpol), a C-band dual-polarisation research radar located at Gunn Point near Darwin, in Australia's northern Territory. 


## Setup

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import shutil
import thuner.data as data
import thuner.option as option
import thuner.track.track as track
import thuner.visualize as visualize
import thuner.analyze as analyze
import thuner.default as default
import thuner.attribute as attribute
import thuner.parallel as parallel

In [ ]:
# Parent directory for saving outputs
base_local = Path.home() / "THUNER_output"

output_parent = base_local / "runs/cpol/geographic"
options_directory = output_parent / "options"
visualize_directory = output_parent / "visualize"

# Remove the output parent directory if it already exists
if output_parent.exists():
    shutil.rmtree(output_parent)

## Geographic Coordinates

CPOL level 1b data is provided in cartesian coordinates. We can convert this data to 
geographic coordinates on the fly by specifying default grid options. We will also save
this converted data to disk for use later.

In [ ]:
# Create the dataset options
start = "2005-11-13T14:00:00"
end = "2005-11-13T19:00:00"
times_dict = {"start": start, "end": end}
cpol_options = data.aura.CPOLOptions(**times_dict, converted_options={"save": True})
era5_dict = {"latitude_range": [-14, -10], "longitude_range": [129, 133]}
era5_pl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
datasets=[cpol_options, era5_pl_options, era5_sl_options]
data_options = option.data.DataOptions(datasets=datasets)
data_options.to_yaml(options_directory / "data.yml")

# Create the grid_options
grid_options = option.grid.GridOptions()
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options
track_options = default.track(dataset_name="cpol")
# Modify the default track options to demonstrate the tracking of both convective 
# objects, and mesoscale convective systems, which are built out of convective, middle 
# and stratiform echo objects, within the same THUNER run. We will use a larger
# minimum size for the convective objects, as too many very small objects confuses the
# matching algorithm.
core = attribute.core.default_tracked()
attributes = option.attribute.Attributes(name="convective", attribute_types=[core])
track_options.levels[0].object_by_name("convective").attributes = attributes
tint_tracking = option.track.TintOptions(search_margin=5)
track_options.levels[0].object_by_name("convective").tracking = tint_tracking
mask_options = option.track.MaskOptions(save=True)
track_options.levels[0].object_by_name("convective").mask_options = mask_options
track_options.levels[0].object_by_name("convective").detection.min_area = 64
track_options.levels[0].object_by_name("convective").detection.altitudes
track_options.levels[0].object_by_name("convective").revalidate()
track_options.levels[0].revalidate()
track_options.to_yaml(options_directory / "track.yml")

# Create the visualize_options
kwargs = {"visualize_directory": visualize_directory, "objects": ["convective", "mcs"]}
visualize_options = default.runtime(**kwargs)
visualize_options.to_yaml(options_directory / "visualize.yml")

We can now perform our tracking run; note the run will be slow as we are generating runtime figures for both convective and MCS objects, and not using parallelization. To make the run go much faster, set `visualize_options = None` and use the the parallel tracking function.

In [ ]:
times = data.generate_times(data_options.dataset_by_name("cpol"))
args = [times, data_options, grid_options, track_options, visualize_options]
# parallel.track(*args, output_directory=output_parent)
track.track(*args, output_directory=output_parent)

In [ ]:
analysis_options = analyze.mcs.AnalysisOptions()
analysis_options.to_yaml(options_directory / "analysis.yml")
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent, analysis_options)

In [ ]:
figure_name = "mcs_attributes"
kwargs = {"style": "presentation", "attributes": ["velocity", "offset"]}
figure_options = option.visualize.HorizontalAttributeOptions(name=figure_name, **kwargs)

args = [output_parent, start, end, figure_options]
args_dict = {"parallel_figure": True, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

## Cartesian Coordinates

Because the CPOL radar domains are small (150 km radii), it is reasonable to perform 
tracking in Cartesian coordinates. This should make the run faster as we are no longer 
performing conversions on the fly. We will also switch off the runtime figure 
generation.

In [ ]:
output_parent = base_local / "runs/cpol/cartesian"
options_directory = output_parent / "options"
options_directory.mkdir(parents=True, exist_ok=True)

if output_parent.exists():
    shutil.rmtree(output_parent)

grid_options = option.grid.GridOptions(name="cartesian", regrid=False)
grid_options.to_yaml(options_directory / "grid.yml")
data_options.to_yaml(options_directory / "data.yml")
track_options.to_yaml(options_directory / "track.yml")
visualize_options = None

times = data.generate_times(data_options.dataset_by_name("cpol"))
args = [times, data_options, grid_options, track_options, visualize_options]
kwargs = {"output_directory": output_parent, "dataset_name": "cpol"}
parallel.track(*args, **kwargs, debug_mode=True)

In [ ]:
analysis_options = analyze.mcs.AnalysisOptions()
analysis_options.to_yaml(options_directory / "analysis.yml")
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent, analysis_options)

In [ ]:
figure_name = "mcs_attributes"
kwargs = {"style": "presentation", "attributes": ["velocity", "offset"]}
figure_options = option.visualize.HorizontalAttributeOptions(name=figure_name, **kwargs)

args = [output_parent, start, end, figure_options]
args_dict = {"parallel_figure": True, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)